# Spark 101

In this lesson we will cover the basics of working with spark dataframes, and
show how spark dataframes are different from the pandas dataframes we have
been working with.

While spark dataframes might superficially look like pandas dataframes, and
even share some of the same methods and syntax, it is important to keep in
mind they are 2 seperate types of objects, and, while spark and pandas code
might look superficially similar, it tends to be semantically very different.

We'll begin by creating the spark session:

In [1]:
import pyspark

In [2]:
# create a pyspark sesion:
spark = pyspark.sql.SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/18 14:05:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark

## Creating Dataframes

Spark can convert any pandas dataframe into a spark dataframe with a simple
method call. For this lesson, we will use this functionality to demonstrate
the differences between spark and pandas dataframes and explore how to work
with spark dataframes.

In [5]:
import pandas as pd
import numpy as np

np.random.seed(1349)
pandas_dataframe = pd.DataFrame(
    dict(n=np.arange(20), group=np.random.choice(list('abc'), 20))
)
pandas_dataframe.head()

,n,group
0,0,c
1,1,c
2,2,b
3,3,a
4,4,c


Here we start with a simple pandas dataset, and now we will convert it to a
spark dataframe:

In [6]:
# use the spark session itself, which we called spark, to create the dataframe
df = spark.createDataFrame(pandas_dataframe)

In [7]:
df

DataFrame[n: bigint, group: string]

Notice that, while we do see the column names, we don't see the data in the
dataframe like we would with a pandas dataframe. This is because spark is
*lazy*, in that it won't show us values until it has to. For the purposes of
looking at the first few rows of our data, we can use the `.show` method.

In [8]:
type(df)

pyspark.sql.dataframe.DataFrame

In [9]:
# df.head() doesn't work with spark
df.show(5)

+---+-----+
|  n|group|
+---+-----+
|  0|    c|
|  1|    c|
|  2|    b|
|  3|    a|
|  4|    c|
+---+-----+
only showing top 5 rows



Like pandas dataframes, spark dataframes have a .describe method:

In [10]:
df.describe().show()
# using df.describe() creates the description
# adding .show actually displays the description

+-------+-----------------+-----+
|summary|                n|group|
+-------+-----------------+-----+
|  count|               20|   20|
|   mean|              9.5| null|
| stddev|5.916079783099616| null|
|    min|                0|    a|
|    max|               19|    c|
+-------+-----------------+-----+



Which, also like pandas, returns another dataframe. However, since this is a
spark dataframe, we have to explicitly show it.

In [11]:
from pydataset import data
mpg = data('mpg')

In [12]:
mpg = spark.createDataFrame(mpg)

By default spark will show the first 20 rows, but we can specify how many we
want by passing a number to `.show`.

Let's use some different data so that we have a more robust dataset:

In [13]:
df.describe().show(2)

+-------+---+-----+
|summary|  n|group|
+-------+---+-----+
|  count| 20|   20|
|   mean|9.5| null|
+-------+---+-----+
only showing top 2 rows



In [14]:
mpg.describe().show()

23/05/18 14:05:46 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+------------+-----------------+------------------+-----------------+-----------------+----------+---+------------------+-----------------+----+-------+
|summary|manufacturer|            model|             displ|             year|              cyl|     trans|drv|               cty|              hwy|  fl|  class|
+-------+------------+-----------------+------------------+-----------------+-----------------+----------+---+------------------+-----------------+----+-------+
|  count|         234|              234|               234|              234|              234|       234|234|               234|              234| 234|    234|
|   mean|        null|             null| 3.471794871794872|           2003.5|5.888888888888889|      null|4.0|16.858974358974358|23.44017094017094|null|   null|
| stddev|        null|             null|1.2919590310839348|4.509646313320436|1.611534484684289|      null|0.0| 4.255945678889394|5.954643441166448|null|   null|
|    min|        audi|      4runne

Let's look at another difference from pandas:

In [15]:
# looking at a subset of columns:
# pandas: df[[''model', 'displ', 'cyl']]
# here we will use select
mpg.select(mpg.model, mpg.displ, mpg.cyl).show(5)

+-----+-----+---+
|model|displ|cyl|
+-----+-----+---+
|   a4|  1.8|  4|
|   a4|  1.8|  4|
|   a4|  2.0|  4|
|   a4|  2.0|  4|
|   a4|  2.8|  6|
+-----+-----+---+
only showing top 5 rows



While this expression would produce a Series of values from a pandas
dataframe, for a spark dataframe this produces a Column object, which is an
object that represents a vertical slice of a dataframe, but does not contain
the data itself.

One way to use our column objects is to use them in combination with the
`.select` method. `.select` is very powerful, and lets us specify what data we
want to see in the resulting dataframe.

Again, notice that we don't see any data, instead we see the new dataframe
that is produced. To see the actual data, we'll again need to use `.show`

In [ ]:
# .select will give us the set of columns we want to see
# we can reference the columns with dot notation just like pandas Series
# and if we want to see the results of our action, we can chain a .show()
# at the end of the command


In [17]:
mpg.select(mpg.hwy, mpg.cty) # doesn't show us anythong

DataFrame[hwy: bigint, cty: bigint]

In [19]:
mpg.select(mpg.hwy, mpg.cty).show(5)

+---+---+
|hwy|cty|
+---+---+
| 29| 18|
| 29| 21|
| 31| 20|
| 30| 21|
| 26| 16|
+---+---+
only showing top 5 rows



Our column objects support a numer of operations, including the arithmetic
operators:

In [20]:
mpg.hwy + 1

Column<'(hwy + 1)'>

Here we get back a column that represents the values from the original `hwy`
column with 1 added to them. To actually see this data, we'd need to select it
and show the dataframe.

In [21]:
mpg.select(mpg.hwy, mpg.hwy+1).show(5)

+---+---------+
|hwy|(hwy + 1)|
+---+---------+
| 29|       30|
| 29|       30|
| 31|       32|
| 30|       31|
| 26|       27|
+---+---------+
only showing top 5 rows



Once we have a column object, we can use the `.alias` method to rename it:

In [23]:
mpg.select(mpg.hwy.alias('highway'), (mpg.hwy+1).alias('highway_plus_1')).show(5)

+-------+--------------+
|highway|highway_plus_1|
+-------+--------------+
|     29|            30|
|     29|            30|
|     31|            32|
|     30|            31|
|     26|            27|
+-------+--------------+
only showing top 5 rows



we can also store column objects in variables and reference them

In [24]:
# col1: the hwy column from the mpg dataframe, aliased as highway_mileage
# col2: the calculation of the highway column divided by two, aliased as highway_mileage_halved
col1 = mpg.hwy.alias('highway_mileage')
col2 = (mpg.hwy /2).alias('highway_mileage_halved')

In [25]:
col2

Column<'(hwy / 2) AS highway_mileage_halved'>

In [26]:
mpg.select(col1, col2).show(3)

+---------------+----------------------+
|highway_mileage|highway_mileage_halved|
+---------------+----------------------+
|             29|                  14.5|
|             29|                  14.5|
|             31|                  15.5|
+---------------+----------------------+
only showing top 3 rows



## Other ways to create columns

In addition to the syntax we've seen above, we can create columns with the
`col` and `expr` functions from `pyspark.sql.functions` module.

### `col`

In [28]:
# some pyspark functions have the same name as vanila python functions
# so we dont override the base functions import like this
from pyspark.sql import functions as F

In [29]:
F.col('hwy')

Column<'hwy'>

In [30]:
# mpg.select(mpg.hwy).show(5)
mpg.select('hwy').show(5)

+---+
|hwy|
+---+
| 29|
| 29|
| 31|
| 30|
| 26|
+---+
only showing top 5 rows



In [31]:
mpg.select(F.col('hwy')).show(5)

+---+
|hwy|
+---+
| 29|
| 29|
| 31|
| 30|
| 26|
+---+
only showing top 5 rows



We can mix and match the syntax we use, and the column object produced by the
`col` function is the same as the the previous column object we saw.

Here we create a variable named `avg_column` that represents the average of the
highway and city mileage of each vehicle. This variable is created by using
the `col` function to produce pyspark Column objects and using the arithmetic
operators to combine them.

Next we select the original highway and city mileage columns, in addition to
our new average mileage column. We demonstrate the `col` function to select
the `hwy` column and refer to the city mileage column with the `df.cty`
syntax we saw previously. We also give all of our columns more readable
aliases before showing the resulting dataframe.

In [38]:
avg_column = (2/(1 / (F.col('hwy')) + (1/ F.col('cty')))).alias('avg_mileage')

In [39]:
avg_column

Column<'(2 / ((1 / hwy) + (1 / cty))) AS avg_mileage'>

In [40]:
mpg.select(avg_column).show(5)

+------------------+
|       avg_mileage|
+------------------+
| 22.21276595744681|
|             24.36|
|24.313725490196077|
| 24.70588235294118|
| 19.80952380952381|
+------------------+
only showing top 5 rows



In [41]:
arith_avg = ((mpg.hwy + mpg.cty) / 2).alias('arith_mean')

In [46]:
mpg.select(F.round(avg_column,2), arith_avg).show(5) # this removes the alias

+------------------------------------------------------+----------+
|round((2 / ((1 / hwy) + (1 / cty))) AS avg_mileage, 2)|arith_mean|
+------------------------------------------------------+----------+
|                                                 22.21|      23.5|
|                                                 24.36|      25.0|
|                                                 24.31|      25.5|
|                                                 24.71|      25.5|
|                                                 19.81|      21.0|
+------------------------------------------------------+----------+
only showing top 5 rows



In [47]:
mpg.select(F.round(avg_column,2).alias('rounded_harmonic'), arith_avg).show(5)

+----------------+----------+
|rounded_harmonic|arith_mean|
+----------------+----------+
|           22.21|      23.5|
|           24.36|      25.0|
|           24.31|      25.5|
|           24.71|      25.5|
|           19.81|      21.0|
+----------------+----------+
only showing top 5 rows



In [ ]:
# from mpg,
# select the column hwy, alias it
# select the column cty, alias it,
# select the column avg_column, alias it


### `expr`

The `expr` function is more powerful than `col`. It does everything `col` does
and more. `expr` returns the same type of column object, but allows us to
express manipulations to the column within the string that defines the column.

In [49]:
# my_col = F.expr(mpg.hwy + 1) this doesn't work
my_col = F.expr('hwy + 1')

In [53]:
# mpg.select(F.col('hwy + 1')).show(5) this doesn't work

mpg.select(my_col).show(5)

In [55]:
mpg.select(
    F.expr('hwy'),# the same as `col`
    F.expr('hwy + 1'),# an arithmetic expression
    F.expr('hwy + 2 AS highway_plus_2')# using an alias
).show(5)

+---+---------+--------------+
|hwy|(hwy + 1)|highway_plus_2|
+---+---------+--------------+
| 29|       30|            31|
| 29|       30|            31|
| 31|       32|            33|
| 30|       31|            32|
| 26|       27|            28|
+---+---------+--------------+
only showing top 5 rows



Note that all the columns created below are identical, and which syntax to use
is merely a style choice.

In [57]:
mpg.select(
    F.expr('hwy').alias('hi_way'),# the same as `col`
    F.expr('hwy + 1 as highway_plus_1'),# an arithmetic expression
    F.expr('hwy + 2 AS highway_plus_2')# using an alias
).show(5)

+------+--------------+--------------+
|hi_way|highway_plus_1|highway_plus_2|
+------+--------------+--------------+
|    29|            30|            31|
|    29|            30|            31|
|    31|            32|            33|
|    30|            31|            32|
|    26|            27|            28|
+------+--------------+--------------+
only showing top 5 rows



## Spark SQL

As we've seen through the column definitions, spark is very flexible and
allows us many different ways to express ourselves. Another way that is fairly
different than what we've seen above is through **spark SQL**, which lets us
write SQL queries against our spark dataframes.

In order to start using spark SQL, we'll first "register" the table with
spark:

In [59]:
# spark.sql('''
# SELECT cty FROM mpg
# ''').show(5)
# this gives us breakage since we haven't registered mpg with spark

In [68]:
mpg.createOrReplaceTempView('mpg')

Now we can write a sql query against the `mpg` table:

In [66]:
spark.sql('''
SELECT cty FROM mpg
''').show(5)

+---+
|cty|
+---+
| 18|
| 21|
| 20|
| 21|
| 16|
+---+
only showing top 5 rows



Notice that the resulting value is another dataframe. As we know, in order to
view the values in a dataframe, we need to use `.show`

In [71]:
spark.sql('''
SELECT hwy, ((hwy + cty) / 2) AS arith_avg
FROM mpg
''').show(5)

+---+---------+
|hwy|arith_avg|
+---+---------+
| 29|     23.5|
| 29|     25.0|
| 31|     25.5|
| 30|     25.5|
| 26|     21.0|
+---+---------+
only showing top 5 rows



It is worth noting that all of the methods for creating / manipulating
dataframes outlined above are the same in terms of performance as well. All of
the resulting dataframes get turned into the same spark code that gets
executed on the JVM, so it really is just a style choice as to which to use.

## Type Casting

We can view the types of the column in our dataframe in one of two ways:

In [72]:

mpg.dtypes

[('manufacturer', 'string'),
 ('model', 'string'),
 ('displ', 'double'),
 ('year', 'bigint'),
 ('cyl', 'bigint'),
 ('trans', 'string'),
 ('drv', 'string'),
 ('cty', 'bigint'),
 ('hwy', 'bigint'),
 ('fl', 'string'),
 ('class', 'string')]

In [73]:
# .printSchema() is similar to a .info() call
mpg.printSchema()

root
 |-- manufacturer: string (nullable = true)
 |-- model: string (nullable = true)
 |-- displ: double (nullable = true)
 |-- year: long (nullable = true)
 |-- cyl: long (nullable = true)
 |-- trans: string (nullable = true)
 |-- drv: string (nullable = true)
 |-- cty: long (nullable = true)
 |-- hwy: long (nullable = true)
 |-- fl: string (nullable = true)
 |-- class: string (nullable = true)



In [77]:
# pandas: .astype(int)

mpg.select(
    mpg.hwy.alias('highway'),
    mpg.hwy.cast('string') # tying to cast as 'str' will break, need to use 'string'
    ).show(5)

+-------+---+
|highway|hwy|
+-------+---+
|     29| 29|
|     29| 29|
|     31| 31|
|     30| 30|
|     26| 26|
+-------+---+
only showing top 5 rows



Both provide the same information.

To convert from one type to another, we can use the `.cast` method on a
column.

In [78]:
mpg.select(
    mpg.hwy.alias('highway'),
    mpg.hwy.cast('string') # tying to cast as 'str' will break, need to use 'string'
    ).printSchema()

root
 |-- highway: long (nullable = true)
 |-- hwy: string (nullable = true)



Note that if a value is not able to be converted, it will be replaced with
null:

In [79]:
mpg.select(
    mpg.model.cast('int')).show(5)
# attempting to convert the string variable 'model' to int results in nulls

+-----+
|model|
+-----+
| null|
| null|
| null|
| null|
| null|
+-----+
only showing top 5 rows



## Basic Built-in Functions

We've used the `col` and `expr` functions, but there are many other functions
within the `pyspark.sql.functions` module, all of which operate on pyspark
dataframe columns. Here we'll demonstrate several:

- `concat`: to concatenate strings
- `sum`: to sum a group
- `avg`: to take the average of a group
- `min`: to find the minimum
- `max`: to find the maximum

**_Note that importing the `sum` function directly will override the built-in
`sum` function._** This means you will get an error if you try to sum a list
of numbers, because `sum` will refernce the pyspark `sum` function, which
works with pyspark dataframe columns, while the built-in `sum` function works
with lists of numbers. The same holds true for the built in `min` and `max`
functions.

In [83]:
mpg.select(
    (F.sum(mpg.hwy) / F.count(mpg.hwy)).alias('avg_hwy'),
    (F.avg(F.col('hwy'))),
).show(5)

+-----------------+-----------------+
|          avg_hwy|         avg(hwy)|
+-----------------+-----------------+
|23.44017094017094|23.44017094017094|
+-----------------+-----------------+



In [ ]:
# Note: The pyspark avg and mean functions are aliases of eachother


!!!tip "`pyspark` imports"
    In this lesson we will explicitly import any functions from `pyspark.sql.functions` that we use, but it very common to see something like:
    
    ```
    from pyspark.sql.functions import *
    ```
    
    which will import *all* of the functions from the `pyspark.sql.functions` module.

In [ ]:
# from pyspark.sql.functions import lit

In [86]:
# mpg.select(
#     mpg.manufacturer,
#     mpg.model,
#     F.concat(F.col('model'), ' : ',F.expr('manufacturer'))
# ).show(5)

In [ ]:
# we can chain an alias on a concat function call
# we need to cast cylinders with a lit() function call to cast it as a string literal


In order to use a string literal as part of our select, we'll need to use the
`lit` function, otherwise spark will try to resolve our string as a column.

In [90]:
# need to include F.lit() around the literal string we are trying to add
mpg.select(
    mpg.manufacturer,
    mpg.model,
    (F.concat(F.col('model'), F.lit(': '),F.expr('manufacturer')).alias('smashed'))
).show(5)

+------------+-----+--------+
|manufacturer|model| smashed|
+------------+-----+--------+
|        audi|   a4|a4: audi|
|        audi|   a4|a4: audi|
|        audi|   a4|a4: audi|
|        audi|   a4|a4: audi|
|        audi|   a4|a4: audi|
+------------+-----+--------+
only showing top 5 rows



Here we select the concatenation of the number of cylinders (the value from
the `cyl` column) and the string literal " cylinders".

## More `pyspark` Functions for String Manipulation

Let's take a look at a couple more functions for string manipulation.

In [ ]:
# from pyspark.sql.functions import regexp_extract, regexp_replace

In order to demonstrate these functions we'll create a dataframe with some text data.

In [91]:
textdf = spark.createDataFrame(
    pd.DataFrame(
        {
            "address": [
                "600 Navarro St ste 600, San Antonio, TX 78205",
                "3130 Broadway St, San Antonio, TX 78209",
                "303 Pearl Pkwy, San Antonio, TX 78215",
                "1255 SW Loop 410, San Antonio, TX 78227",
            ]
        }
    )
)

textdf.show()

+--------------------+
|             address|
+--------------------+
|600 Navarro St st...|
|3130 Broadway St,...|
|303 Pearl Pkwy, S...|
|1255 SW Loop 410,...|
+--------------------+



In [92]:
# to not truncate the text
textdf.show(truncate=False)

+---------------------------------------------+
|address                                      |
+---------------------------------------------+
|600 Navarro St ste 600, San Antonio, TX 78205|
|3130 Broadway St, San Antonio, TX 78209      |
|303 Pearl Pkwy, San Antonio, TX 78215        |
|1255 SW Loop 410, San Antonio, TX 78227      |
+---------------------------------------------+



The `regexp_extract` function lets us specify a regular expression with at least one capture group, and create a new column based on the contents of a capture group.

In [99]:
# in my select call:
# address, the column as it stands
# regexp_extract, acting as a re.extract, aliasing that with a .alias method 
textdf.select(
    "address",
    F.regexp_extract('address', r'^(\d+)', 1).alias("street_no"),
    F.regexp_extract('address', r'^\d+\s([\w\s]+?),', 1).alias("street"),
).show(truncate=False)

+---------------------------------------------+---------+------------------+
|address                                      |street_no|street            |
+---------------------------------------------+---------+------------------+
|600 Navarro St ste 600, San Antonio, TX 78205|600      |Navarro St ste 600|
|3130 Broadway St, San Antonio, TX 78209      |3130     |Broadway St       |
|303 Pearl Pkwy, San Antonio, TX 78215        |303      |Pearl Pkwy        |
|1255 SW Loop 410, San Antonio, TX 78227      |1255     |SW Loop 410       |
+---------------------------------------------+---------+------------------+



In the example above, the first argument to `regexp_extract` is the name of the string column to extract from, the second argument is the regular expression itself, and the last argument specifies which capture group we want to use. If, for example, our regular expression had 2 capture groups in it and we wanted the contents of the 2nd group, we would specify a 2 here.

In addition to `regexp_extract`, `regexp_replace` lets us make substitutions based on a regular expression.

In [102]:
textdf.select(
    "address",
#     F.regexp_replace('subject', 'pattern', 'replacement)
    F.regexp_replace('address', '^.*?,\s', '').alias("city_state_zip"),
).show(truncate=False)

+---------------------------------------------+---------------------+
|address                                      |city_state_zip       |
+---------------------------------------------+---------------------+
|600 Navarro St ste 600, San Antonio, TX 78205|San Antonio, TX 78205|
|3130 Broadway St, San Antonio, TX 78209      |San Antonio, TX 78209|
|303 Pearl Pkwy, San Antonio, TX 78215        |San Antonio, TX 78215|
|1255 SW Loop 410, San Antonio, TX 78227      |San Antonio, TX 78227|
+---------------------------------------------+---------------------+



In our example above, we obtain just the city, state, and zip code of the address by replacing everything up to the first comma with an empty string.

## `.filter` and `.where`

Spark provides two dataframe methods, `.filter` and `.where`, which both allow
us to select a subset of the rows of our dataframe.

In [ ]:
# pandas:
# mpg[mpg.cyl == 4]

In [104]:
mpg.filter(mpg.cyl == 4).show(5)

+------------+----------+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|     model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+----------+-----+----+---+----------+---+---+---+---+-------+
|        audi|        a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|        a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|        a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|        a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|a4 quattro|  1.8|1999|  4|manual(m5)|  4| 18| 26|  p|compact|
+------------+----------+-----+----+---+----------+---+---+---+---+-------+
only showing top 5 rows



In [105]:
mpg.filter(mpg.cyl == 4).where(mpg['class'] == 'subcompact').show(5)
# 'class' is still a specific function and needs to be passed in bracket notation
# or using F.col('class')

+------------+-----+-----+----+---+----------+---+---+---+---+----------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty|hwy| fl|     class|
+------------+-----+-----+----+---+----------+---+---+---+---+----------+
|       honda|civic|  1.6|1999|  4|manual(m5)|  f| 28| 33|  r|subcompact|
|       honda|civic|  1.6|1999|  4|  auto(l4)|  f| 24| 32|  r|subcompact|
|       honda|civic|  1.6|1999|  4|manual(m5)|  f| 25| 32|  r|subcompact|
|       honda|civic|  1.6|1999|  4|manual(m5)|  f| 23| 29|  p|subcompact|
|       honda|civic|  1.6|1999|  4|  auto(l4)|  f| 24| 32|  r|subcompact|
+------------+-----+-----+----+---+----------+---+---+---+---+----------+
only showing top 5 rows



In [ ]:
# from pyspark.sql.functions import when

## When and Otherwise

Similar to an `IF` in Excel, `CASE...WHEN` in SQL, or `np.where` in python,
spark provides a `when` function.

The `when` function lets us specify a condition, and a value to produce if
that condition is true:

In [106]:
from pyspark.sql.functions import when

In [111]:
mpg.select(
    mpg.hwy,
    F.when(mpg.hwy > 27, 'good_mileage').when(mpg.hwy > 26, 'avg_mileage').\
    otherwise('bad_mileage')
).show(10)

+---+--------------------------------------------------------------------------------------------+
|hwy|CASE WHEN (hwy > 27) THEN good_mileage WHEN (hwy > 26) THEN avg_mileage ELSE bad_mileage END|
+---+--------------------------------------------------------------------------------------------+
| 29|                                                                                good_mileage|
| 29|                                                                                good_mileage|
| 31|                                                                                good_mileage|
| 30|                                                                                good_mileage|
| 26|                                                                                 bad_mileage|
| 26|                                                                                 bad_mileage|
| 27|                                                                                 avg_mileage|
| 26|     

In [115]:
spark.sql('''
SELECT hwy,
    CASE
        WHEN hwy > 27 THEN 'good'
        WHEN hwy > 25 THEN 'avg'
        ELSE 'bad'
    END mileage
FROM mpg
''').show(10)

+---+-------+
|hwy|mileage|
+---+-------+
| 29|   good|
| 29|   good|
| 31|   good|
| 30|   good|
| 26|    avg|
| 26|    avg|
| 27|    avg|
| 26|    avg|
| 25|    bad|
| 28|   good|
+---+-------+
only showing top 10 rows



Notice here that if the condition we specified is false, `null` will be
produced. Instead of null, we can specify a value to use if our condition is
false with the `.otherwise` method.

In [ ]:
m

To specify multiple conditions, we can chain `.when` calls. The first
condition that is met will be the value that is used, and if none of the
conditions are met the value specified in the `.otherwise` will be used (or
`null` if you don't provide a `.otherwise`).

Notice here that a car with a `displ` of 1.8 matches both conditions we
specified, but `small` is produced because it is associated with the first
matching condition. For any value between 2 and 3, `medium` will be produced,
and anything larger than 3 will produce `large`.

## Sorting and Ordering

Spark lets us sort the rows in our dataframe by one or multiple columns with
two methods: `.sort`, and `.orderBy`. `.sort` and `.orderBy` are aliases of
each other and do the exact same thing. Like other methods we've seen, `.sort`
takes in a Column object or a string that is the name of a column.

In [116]:
# default sort is ascending
mpg.sort(mpg.cyl).show(10)

+------------+------------+-----+----+---+----------+---+---+---+---+----------+
|manufacturer|       model|displ|year|cyl|     trans|drv|cty|hwy| fl|     class|
+------------+------------+-----+----+---+----------+---+---+---+---+----------+
|        audi|          a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|   compact|
|      subaru| impreza awd|  2.5|1999|  4|manual(m5)|  4| 19| 26|  r|subcompact|
|       honda|       civic|  1.6|1999|  4|manual(m5)|  f| 28| 33|  r|subcompact|
|      subaru|forester awd|  2.5|1999|  4|manual(m5)|  4| 18| 25|  r|       suv|
|      toyota|camry solara|  2.2|1999|  4|  auto(l4)|  f| 21| 27|  r|   compact|
|      subaru|forester awd|  2.5|1999|  4|  auto(l4)|  4| 18| 24|  r|       suv|
|     hyundai|      sonata|  2.4|1999|  4|manual(m5)|  f| 18| 27|  r|   midsize|
|      subaru|forester awd|  2.5|2008|  4|manual(m5)|  4| 20| 27|  r|       suv|
|  volkswagen|       jetta|  2.0|1999|  4|  auto(l4)|  f| 19| 26|  r|   compact|
|      subaru|forester awd| 

By default, values are sorted in ascending order. To sort in descending order,
we can use the `.desc` method on any Column object, or the `desc` function
from `pyspark.sql.functions`.

In [ ]:
# from pyspark.sql.functions import asc, desc

In [118]:
# sort descending
mpg.sort(mpg.cyl.desc()).show(10)

+------------+------------------+-----+----+---+--------+---+---+---+---+------+
|manufacturer|             model|displ|year|cyl|   trans|drv|cty|hwy| fl| class|
+------------+------------------+-----+----+---+--------+---+---+---+---+------+
|        ford|   f150 pickup 4wd|  4.6|2008|  8|auto(l4)|  4| 13| 17|  r|pickup|
|   chevrolet|   k1500 tahoe 4wd|  5.3|2008|  8|auto(l4)|  4| 11| 14|  e|   suv|
|       dodge|       durango 4wd|  4.7|2008|  8|auto(l5)|  4| 13| 17|  r|   suv|
|   chevrolet|   k1500 tahoe 4wd|  5.7|1999|  8|auto(l4)|  4| 11| 15|  r|   suv|
|      nissan|    pathfinder 4wd|  5.6|2008|  8|auto(s5)|  4| 12| 18|  p|   suv|
|   chevrolet|   k1500 tahoe 4wd|  6.5|1999|  8|auto(l4)|  4| 14| 17|  d|   suv|
|       dodge|       durango 4wd|  4.7|2008|  8|auto(l5)|  4|  9| 12|  e|   suv|
|       dodge| dakota pickup 4wd|  4.7|2008|  8|auto(l5)|  4| 14| 19|  r|pickup|
|   chevrolet|c1500 suburban 2wd|  5.7|1999|  8|auto(l4)|  r| 13| 17|  r|   suv|
|       dodge| dakota pickup

To specify sorting by multiple columns, we provide each column as a separate
argument to `.sort`.

In [120]:
mpg.sort(mpg.hwy.desc()).sort(F.col('cty').desc()).show(10)

+------------+----------+-----+----+---+----------+---+---+---+---+----------+
|manufacturer|     model|displ|year|cyl|     trans|drv|cty|hwy| fl|     class|
+------------+----------+-----+----+---+----------+---+---+---+---+----------+
|  volkswagen|new beetle|  1.9|1999|  4|manual(m5)|  f| 35| 44|  d|subcompact|
|  volkswagen|     jetta|  1.9|1999|  4|manual(m5)|  f| 33| 44|  d|   compact|
|  volkswagen|new beetle|  1.9|1999|  4|  auto(l4)|  f| 29| 41|  d|subcompact|
|      toyota|   corolla|  1.8|2008|  4|manual(m5)|  f| 28| 37|  r|   compact|
|       honda|     civic|  1.6|1999|  4|manual(m5)|  f| 28| 33|  r|subcompact|
|      toyota|   corolla|  1.8|1999|  4|manual(m5)|  f| 26| 35|  r|   compact|
|       honda|     civic|  1.8|2008|  4|manual(m5)|  f| 26| 34|  r|subcompact|
|      toyota|   corolla|  1.8|2008|  4|  auto(l4)|  f| 26| 35|  r|   compact|
|       honda|     civic|  1.6|1999|  4|manual(m5)|  f| 25| 32|  r|subcompact|
|       honda|     civic|  1.8|2008|  4|  auto(l5)| 

Here we will first reverse alphabetically by number of cylinders from lowest to highest, then by the vehicle's highway
mileage, from greatest to smallest.

## Grouping and Aggregating

To aggregate our data by group, we can use the `.groupBy` method. Like with
`.select`, we can pass either Column objects or strings that are column names
to `.groupBy`. All of the expressions below are equivalent.

In [121]:
mpg.groupBy(mpg.cyl)
mpg.groupBy(F.col('cyl'))
mpg.groupBy('cyl')

Once the data is grouped, we need to specify an aggregation. We can use one of
the aggregate functions we imported earlier, alond with a column:

In [124]:
# we can use .agg
mpg.groupby(mpg.cyl).agg(F.avg('hwy')).sort('cyl').show(10)

+---+-----------------+
|cyl|         avg(hwy)|
+---+-----------------+
|  4|28.80246913580247|
|  5|            28.75|
|  6|22.82278481012658|
|  8|17.62857142857143|
+---+-----------------+



To group by multiple columns, pass each of the columns a a separate argument
to `.groupBy` (Note that this is different from pandas, where we would need to
pass a list).

In [125]:
mpg.groupby(mpg.cyl).agg(F.avg('hwy'), F.avg(F.col('cty'))).sort('cyl').show(10)

+---+-----------------+------------------+
|cyl|         avg(hwy)|          avg(cty)|
+---+-----------------+------------------+
|  4|28.80246913580247|21.012345679012345|
|  5|            28.75|              20.5|
|  6|22.82278481012658| 16.21518987341772|
|  8|17.62857142857143|12.571428571428571|
+---+-----------------+------------------+



In [127]:

mpg.groupby(mpg.cyl, mpg['class']).agg(F.avg('hwy'), F.avg(F.col('cty'))).\
    sort('cyl').show(10)

+---+----------+------------------+------------------+
|cyl|     class|          avg(hwy)|          avg(cty)|
+---+----------+------------------+------------------+
|  4|   compact|          29.46875|            21.375|
|  4|   midsize|           29.1875|              20.5|
|  4|   minivan|              24.0|              18.0|
|  4|subcompact| 30.80952380952381|22.857142857142858|
|  4|       suv|             23.75|              18.0|
|  4|    pickup|20.666666666666668|              16.0|
|  5|   compact|              29.0|              21.0|
|  5|subcompact|              28.5|              20.0|
|  6|   compact|25.307692307692307|16.923076923076923|
|  6|   midsize| 26.26086956521739|17.782608695652176|
+---+----------+------------------+------------------+
only showing top 10 rows



In addition to `.groupBy`, we can use `.rollup`, which will do the same
aggregations, but will also include the overall total:

In [128]:
mpg.rollup(mpg.cyl, mpg['class']).agg(F.avg('hwy'), F.avg(F.col('cty'))).\
    sort('cyl').show(10)

+----+----------+------------------+------------------+
| cyl|     class|          avg(hwy)|          avg(cty)|
+----+----------+------------------+------------------+
|null|      null| 23.44017094017094|16.858974358974358|
|   4|   midsize|           29.1875|              20.5|
|   4|      null| 28.80246913580247|21.012345679012345|
|   4|   compact|          29.46875|            21.375|
|   4|   minivan|              24.0|              18.0|
|   4|subcompact| 30.80952380952381|22.857142857142858|
|   4|       suv|             23.75|              18.0|
|   4|    pickup|20.666666666666668|              16.0|
|   5|   compact|              29.0|              21.0|
|   5|      null|             28.75|              20.5|
+----+----------+------------------+------------------+
only showing top 10 rows



Here the null value in `cyl` indicates the total count.

And in the example above, the null row represents the overall average highway
mileage.

In [131]:
mpg.filter(mpg.cyl < 5).rollup(mpg.cyl, mpg['class']).agg(F.avg('hwy'), 
                                                          F.avg(F.col('cty'))).\
    sort('cyl').show(10)

+----+----------+------------------+------------------+
| cyl|     class|          avg(hwy)|          avg(cty)|
+----+----------+------------------+------------------+
|null|      null| 28.80246913580247|21.012345679012345|
|   4|      null| 28.80246913580247|21.012345679012345|
|   4|   compact|          29.46875|            21.375|
|   4|   midsize|           29.1875|              20.5|
|   4|   minivan|              24.0|              18.0|
|   4|subcompact| 30.80952380952381|22.857142857142858|
|   4|       suv|             23.75|              18.0|
|   4|    pickup|20.666666666666668|              16.0|
+----+----------+------------------+------------------+



## Crosstabs and Pivot Tables

In addition to groupby, spark provides a couple other ways to do aggregation.
One of which is `.crosstab`. This is very similary to pandas `.crosstab`
function, in that it calculates the number of occurances of each unique value
from the two passed columns:

In [132]:
mpg.crosstab('cyl', 'class').show()

+---------+-------+-------+-------+-------+------+----------+---+
|cyl_class|2seater|compact|midsize|minivan|pickup|subcompact|suv|
+---------+-------+-------+-------+-------+------+----------+---+
|        8|      5|      0|      2|      0|    20|         5| 38|
|        5|      0|      2|      0|      0|     0|         2|  0|
|        6|      0|     13|     23|     10|    10|         7| 16|
|        4|      0|     32|     16|      1|     3|        21|  8|
+---------+-------+-------+-------+-------+------+----------+---+



`.crosstab` simply does counts, if we want a different aggregation, we can use
`.pivot`. For example, to find the average highway mileage for each
combination of car class and number of cylinders, we could write the
following:

In [134]:
# mpg.pivot('cyl', 'class').show() # dataframe has no pivot function

In [135]:
mpg.groupby('class').pivot('cyl').mean('hwy').show()

+----------+------------------+----+------------------+------------------+
|     class|                 4|   5|                 6|                 8|
+----------+------------------+----+------------------+------------------+
|subcompact| 30.80952380952381|28.5|24.714285714285715|              21.6|
|   compact|          29.46875|29.0|25.307692307692307|              null|
|   minivan|              24.0|null|              22.2|              null|
|       suv|             23.75|null|              18.5|16.789473684210527|
|   midsize|           29.1875|null| 26.26086956521739|              24.0|
|    pickup|20.666666666666668|null|              17.9|              15.8|
|   2seater|              null|null|              null|              24.8|
+----------+------------------+----+------------------+------------------+



Here the unique values from the column we group by will be the rows in the
resulting dataframe, and the unique values from the column we pivot on will
become the columns. The values in each cell will be equal to the aggregation
we specified over the group of values defined by the intersection of the rows
and the columns.

## Handling Missing Data

Let's take a look at how spark handles missing data. First we'll create a dataframe that has a few missing values:

In [136]:
df = spark.createDataFrame(
    pd.DataFrame(
        {"x": [1, 2, np.nan, 4, 5, np.nan], "y": [np.nan, 0, 0, 3, 1, np.nan]}
    )
)
df.show()

+---+---+
|  x|  y|
+---+---+
|1.0|NaN|
|2.0|0.0|
|NaN|0.0|
|4.0|3.0|
|5.0|1.0|
|NaN|NaN|
+---+---+



Spark provides two main ways to deal with missing values:

- `.fill`: to replace missing values with a specified value
- `.drop`: to drop rows containing missing values

Both methods are accessed through the `.na` property. We'll look at some examples below:

In [139]:
# df.na.fill?
df.na.fill(0).show()

+---+---+
|  x|  y|
+---+---+
|1.0|0.0|
|2.0|0.0|
|0.0|0.0|
|4.0|3.0|
|5.0|1.0|
|0.0|0.0|
+---+---+



In [140]:
# fill nans only in the y column
df.na.fill(0, subset='y').show()

+---+---+
|  x|  y|
+---+---+
|1.0|0.0|
|2.0|0.0|
|NaN|0.0|
|4.0|3.0|
|5.0|1.0|
|NaN|0.0|
+---+---+



In [141]:
df.na.fill(0, subset='y').na.fill(1, subset='x').show()

+---+---+
|  x|  y|
+---+---+
|1.0|0.0|
|2.0|0.0|
|1.0|0.0|
|4.0|3.0|
|5.0|1.0|
|1.0|0.0|
+---+---+



For both methods, we can specify that we only want to fill or drop values in a specific column with a second argument:

Notice that above the na values in the `x` column were filled with 0, but the na values in y were left alone.

In the example above, the rows that had an na value for the y column were dropped, but the rows with na values for only the x column are still present.

In [142]:
df.na.drop(subset='x').show()

+---+---+
|  x|  y|
+---+---+
|1.0|NaN|
|2.0|0.0|
|4.0|3.0|
|5.0|1.0|
+---+---+



In [143]:
df2 = df.na.drop(subset='x')

In [144]:
df2.show()

+---+---+
|  x|  y|
+---+---+
|1.0|NaN|
|2.0|0.0|
|4.0|3.0|
|5.0|1.0|
+---+---+



## withColumn:

In [146]:
mpg.withColumn('highway_again', F.col('hwy')).show(5)

+------------+-----+-----+----+---+----------+---+---+---+---+-------+-------------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|highway_again|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+-------------+
|        audi|   a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|           29|
|        audi|   a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|           29|
|        audi|   a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|           31|
|        audi|   a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|           30|
|        audi|   a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|           26|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+-------------+
only showing top 5 rows



In [148]:
mpg.withColumn('highway_again', F.col('hwy')).drop('hwy').show(5)

+------------+-----+-----+----+---+----------+---+---+---+-------+-------------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty| fl|  class|highway_again|
+------------+-----+-----+----+---+----------+---+---+---+-------+-------------+
|        audi|   a4|  1.8|1999|  4|  auto(l5)|  f| 18|  p|compact|           29|
|        audi|   a4|  1.8|1999|  4|manual(m5)|  f| 21|  p|compact|           29|
|        audi|   a4|  2.0|2008|  4|manual(m6)|  f| 20|  p|compact|           31|
|        audi|   a4|  2.0|2008|  4|  auto(av)|  f| 21|  p|compact|           30|
|        audi|   a4|  2.8|1999|  6|  auto(l5)|  f| 16|  p|compact|           26|
+------------+-----+-----+----+---+----------+---+---+---+-------+-------------+
only showing top 5 rows



## More Dataframe Manipulation Examples

Let's take a look at some more examples of working with spark dataframes. For
these examples, we'll be working with a dataset of observations of the
weather in seattle.

In [163]:
from vega_datasets import data

weather = data.seattle_weather().assign(date=lambda df: df.date.astype(str))
weather = spark.createDataFrame(weather)
weather.show(6)

+----------+-------------+--------+--------+----+-------+
|      date|precipitation|temp_max|temp_min|wind|weather|
+----------+-------------+--------+--------+----+-------+
|2012-01-01|          0.0|    12.8|     5.0| 4.7|drizzle|
|2012-01-02|         10.9|    10.6|     2.8| 4.5|   rain|
|2012-01-03|          0.8|    11.7|     7.2| 2.3|   rain|
|2012-01-04|         20.3|    12.2|     5.6| 4.7|   rain|
|2012-01-05|          1.3|     8.9|     2.8| 6.1|   rain|
|2012-01-06|          2.5|     4.4|     2.2| 2.2|   rain|
+----------+-------------+--------+--------+----+-------+
only showing top 6 rows



Let's print out the number of rows and columns in our dataset:

In [150]:
weather.describe().show()
# this gives us info, but what if we want the shape

+-------+----------+-----------------+------------------+-----------------+------------------+-------+
|summary|      date|    precipitation|          temp_max|         temp_min|              wind|weather|
+-------+----------+-----------------+------------------+-----------------+------------------+-------+
|  count|      1461|             1461|              1461|             1461|              1461|   1461|
|   mean|      null| 3.02943189596167|16.439082819986314|8.234770704996578|3.2411362080766604|   null|
| stddev|      null|6.680194322314738| 7.349758097360176|5.023004179961266|1.4378250588746198|   null|
|    min|2012-01-01|              0.0|              -1.6|             -7.1|               0.4|drizzle|
|    max|2015-12-31|             55.9|              35.6|             18.3|               9.5|    sun|
+-------+----------+-----------------+------------------+-----------------+------------------+-------+



In [153]:
weather.count(), len(weather.columns)

(1461, 6)

Let's first find the dates where the data starts and stops:

In [164]:
min_date, max_date = weather.select(F.min(F.col('date')), F.max('date'))
min_date, max_date

(Column<'min(date)'>, Column<'max(date)'>)

Here we use `.select` to select the minimum date and the maximum date.
`.first` returns us the first row of our results, which consists of two value,
and so can be unpacked into the `min_date` and `max_date` variables.

In [160]:
min_date, max_date = weather.select(F.min(F.col('date')), F.max('date')).first()
min_date, max_date

('2012-01-01', '2015-12-31')

Next we will combine the temp max and min columns into a single column,
`temp_avg`.

In [165]:
# withColumn is similar to our df.assign()
weather = weather.withColumn('temp_avg', F.expr('(temp_min + temp_max) / 2')).drop(
    'temp_min', 'temp_max')


In [166]:
weather.show(10)

+----------+-------------+----+-------+------------------+
|      date|precipitation|wind|weather|          temp_avg|
+----------+-------------+----+-------+------------------+
|2012-01-01|          0.0| 4.7|drizzle|               8.9|
|2012-01-02|         10.9| 4.5|   rain| 6.699999999999999|
|2012-01-03|          0.8| 2.3|   rain|              9.45|
|2012-01-04|         20.3| 4.7|   rain| 8.899999999999999|
|2012-01-05|          1.3| 6.1|   rain|              5.85|
|2012-01-06|          2.5| 2.2|   rain|3.3000000000000003|
|2012-01-07|          0.0| 2.3|   rain|               5.0|
|2012-01-08|          0.0| 2.0|    sun|               6.4|
|2012-01-09|          4.3| 3.4|   rain|               7.2|
|2012-01-10|          1.0| 3.4|   rain|3.3499999999999996|
+----------+-------------+----+-------+------------------+
only showing top 10 rows



Now we will calculate the total amount of rainfall for each month. We'll do
this by first creating a month column, then grouping by the month, and
finally, aggregating by taking the sum of the precipitation. To do this we will need to use the `month` function.

In [ ]:
# from pyspark.sql.functions import month, year, quarter

In [167]:
weather.printSchema()

root
 |-- date: string (nullable = true)
 |-- precipitation: double (nullable = true)
 |-- wind: double (nullable = true)
 |-- weather: string (nullable = true)
 |-- temp_avg: double (nullable = true)



In [169]:
# date is being interpreted as a string instead of date
weather.withColumn('month', F.month(F.col('date'))).show(10)

+----------+-------------+----+-------+------------------+-----+
|      date|precipitation|wind|weather|          temp_avg|month|
+----------+-------------+----+-------+------------------+-----+
|2012-01-01|          0.0| 4.7|drizzle|               8.9|    1|
|2012-01-02|         10.9| 4.5|   rain| 6.699999999999999|    1|
|2012-01-03|          0.8| 2.3|   rain|              9.45|    1|
|2012-01-04|         20.3| 4.7|   rain| 8.899999999999999|    1|
|2012-01-05|          1.3| 6.1|   rain|              5.85|    1|
|2012-01-06|          2.5| 2.2|   rain|3.3000000000000003|    1|
|2012-01-07|          0.0| 2.3|   rain|               5.0|    1|
|2012-01-08|          0.0| 2.0|    sun|               6.4|    1|
|2012-01-09|          4.3| 3.4|   rain|               7.2|    1|
|2012-01-10|          1.0| 3.4|   rain|3.3499999999999996|    1|
+----------+-------------+----+-------+------------------+-----+
only showing top 10 rows



In [173]:
weather.withColumn('month', F.month(F.col('date'))).groupby('month').\
    pivot('weather').avg('precipitation').show(10)

+-----+-------+-------------------+-------------------+------------------+-------------------+
|month|drizzle|                fog|               rain|              snow|                sun|
+-----+-------+-------------------+-------------------+------------------+-------------------+
|   12|    0.0|  7.990740740740743| 5.0260869565217385|11.680000000000001|0.43000000000000005|
|    1|    0.0|  3.792105263157894|  6.614285714285714|               8.6| 0.6545454545454544|
|    6|    0.0| 2.8285714285714287|  3.952631578947368|              null|0.21411764705882352|
|    3|    0.0|  9.572222222222223|  4.921621621621622|11.766666666666666|0.21190476190476187|
|    5|    0.0|              5.968| 3.2624999999999997|              null|0.07439024390243902|
|    9|    0.0|              4.625|0.22499999999999998|              null| 0.6985915492957746|
|    4|   0.25|               8.15|  3.429999999999999|               4.6| 0.3950819672131148|
|    8|    0.0|             5.9875|  6.43333333333

In [179]:
weather.withColumn('month', F.month(F.col('date'))).groupby('month', 'weather').\
    avg('precipitation').sort('month').show(10)

+-----+-------+-------------------+
|month|weather| avg(precipitation)|
+-----+-------+-------------------+
|    1|   snow|                8.6|
|    1|    sun| 0.6545454545454544|
|    1|drizzle|                0.0|
|    1|   rain|  6.614285714285714|
|    1|    fog|  3.792105263157894|
|    2|drizzle|                0.0|
|    2|   rain| 3.1725000000000003|
|    2|    sun|0.13333333333333333|
|    2|   snow| 1.9000000000000001|
|    2|    fog|  7.927777777777777|
+-----+-------+-------------------+
only showing top 10 rows



The `.sort` at the end isn't necessary, but presents that data in a friendlier
way.

Let's now take a look at the average temperature for each type of weather in
December 2013:

In [ ]:
# month will rip out the month numeral from the date
# establish a filter df[df.month == 12]
# pass a second filter of df[df.year == 2013]
# aggregate based on weather tag, present average temperature


Here we first have a couple of `.filter` calls in order to restrict our data
to December of 2013. We then group by the weather column, and lastly,
aggregate by taking the average of our `temp_avg` column. The combination of
group by and agg will calculate the average temperature for each unique value
of the `weather` column.

Let's now find out how many days had freezing temperatures in each month of
2013.

One last example, let's calculate the average temperature for each quarter of
each year:

Here we create the `quarter` and `year` columns, then group by these two new columns, and take the average temperature as our aggregate. Lastly, we sort by the year and quarter for presentation purposes.

We could also use a pivot table like this:

Here instead of grouping by two columns, we grouped by the first column and pivoted on the other column.

## Joins

Like pandas and sql, spark has functionality that lets us combine two tabular
datasets, known as a **join**.

We'll start by creating some data that we can join together:

In [180]:
users = spark.createDataFrame(
    pd.DataFrame(
        {
            "id": [1, 2, 3, 4, 5, 6],
            "name": ["bob", "joe", "sally", "adam", "jane", "mike"],
            "role_id": [1, 2, 3, 3, np.nan, np.nan],
        }
    )
)
roles = spark.createDataFrame(
    pd.DataFrame(
        {
            "id": [1, 2, 3, 4],
            "name": ["admin", "author", "reviewer", "commenter"],
        }
    )
)
print("--- users ---")
users.show()
print("--- roles ---")
roles.show()

--- users ---
+---+-----+-------+
| id| name|role_id|
+---+-----+-------+
|  1|  bob|    1.0|
|  2|  joe|    2.0|
|  3|sally|    3.0|
|  4| adam|    3.0|
|  5| jane|    NaN|
|  6| mike|    NaN|
+---+-----+-------+

--- roles ---
+---+---------+
| id|     name|
+---+---------+
|  1|    admin|
|  2|   author|
|  3| reviewer|
|  4|commenter|
+---+---------+



To join two dataframes together, we'll need to call the `.join` method on one
of them and supply the other as an argument. In addition, we'll need to supply
the condition on which we are joining. In our case, we are joining where the
`role_id` column on the users table is equal to the `id` column on the roles
table.

In [181]:
users.join(roles, how='inner', on=users.role_id==roles.id).show()

+---+-----+-------+---+--------+
| id| name|role_id| id|    name|
+---+-----+-------+---+--------+
|  1|  bob|    1.0|  1|   admin|
|  2|  joe|    2.0|  2|  author|
|  3|sally|    3.0|  3|reviewer|
|  4| adam|    3.0|  3|reviewer|
+---+-----+-------+---+--------+



By default, spark will perform an inner join, meaning that records from both
dataframes will have a match with the other. We can also specify either a left
or a right join, which will keep all of the records from either the left or
right side, even if those records don't have a match with the other dataframe.

Notice that examples above have a duplicate `id` column. There are several
ways we could go about dealing with this:

- alias each dataframe + explicitly select columns after joining (this could also be implemented with spark SQL)
- rename duplicated columns before merging
- drop duplicated columns after the merge (`.drop(right.id)`)

## Visualization (or Lack Therof)

Spark does not provide a way to do visualization with their dataframes. To
visualize data from spark, you should use the `.toPandas` method on a spark
dataframe to convert it to a pandas dataframe, then visualize as you normally
would.

!!!warning "Converting to A Pandas Dataframe"
    Converting a spark dataframe to a pandas dataframe will pull all the data into memory, so make sure you have enough available memory to do so.

## Exercises

Using the [repo setup directions](https://ds.codeup.com/fundamentals/git/), setup a new local and remote repository named `spark-exercises`. The local version of your repo should live inside of `~/codeup-data-science`. This repo should be named `spark-exercises`

Save this work in your `spark-exercises` repo. Then add, commit, and push your changes.

Create a jupyter notebook or python script named `spark101` for this exercise.

1. Create a spark data frame that contains your favorite programming languages.

    - The name of the column should be `language`
    - View the schema of the dataframe
    - Output the shape of the dataframe
    - Show the first 5 records in the dataframe

1. Load the `mpg` dataset as a spark dataframe.

    1. Create 1 column of output that contains a message like the one below:

            The 1999 audi a4 has a 4 cylinder engine.

        For each vehicle.

    1. Transform the `trans` column so that it only contains either `manual` or `auto`.

1. Load the `tips` dataset as a spark dataframe.

    1. What percentage of observations are smokers?
    1. Create a column that contains the tip percentage
    1. Calculate the average tip percentage for each combination of sex and smoker.

1. Use the seattle weather dataset referenced in the lesson to answer the questions below.

    - Convert the temperatures to fahrenheit.
    - Which month has the most rain, on average?
    - Which year was the windiest?
    - What is the most frequent type of weather in January?
    - What is the average high and low temperature on sunny days in July in 2013 and 2014?
    - What percentage of days were rainy in q3 of 2015?
    - For each year, find what percentage of days it rained (had non-zero precipitation).